# IMPORT

In [ ]:
pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 구글드라이브 사용
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import re
import time

# 데이터 크롤링

In [ ]:
url_base = 'http://www.booking.com'
url_sub = '/searchresults.ko.html?dest_id=-714191&dest_type=city&checkin=2023-07-31&checkout=2023-08-02&group_adults=2&no_rooms=1&offset='

data = []

for offset in range(0, 276, 25) :
  print(str(offset//25+1) + 'page    - - - - - ing')
  print()
  url = url_base + url_sub + str(offset)

  headers={
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Cookie':'px_init=0; cors_js=1; pxcts=50fe2779-0101-11ee-8933-77634f496d71; _pxvid=50fe17fb-0101-11ee-8933-c6892157769a; bkng_sso_session=e30; OptanonConsent=implicitConsentCountry=nonGDPR&implicitConsentDate=1685681565899; _gcl_au=1.1.380996651.1685681568; bkng_prue=1; _pin_unauth=dWlkPU9HUTJOamcxTmpRdFpURXlOQzAwWm1NMUxXSXlOMll0TkRWbU5EUTRNak5rWkdGag; _scid=3d7dbe98-21c5-466f-bc63-906fea9ac44f; FPID=FPID2.2.nHLjNRnMk4vSNOlYGsPF33zdfmfU%2FML9mjj0o7FqZ2c%3D.1685681565; _sctr=1%7C1685631600000; _ga_A12345=GS1.1.1685681568.1.1.1685681648.0.0.0; bkng_sso_ses=eyJib29raW5nX2dsb2JhbCI6W3siaCI6ImVWSzVQRWwreHlyQkFlTW5hREpwSVlMOEVadG9rVDdScERwWUxpSGJ3ek0iLCJhIjoxfV19; pcm_consent=consentedAt%3D2023-06-08T04%3A06%3A22.650Z%26countryCode%3DKR%26expiresAt%3D2023-12-05T04%3A06%3A22.650Z%26implicit%3Dfalse%26regionCode%3D11%26regulation%3Dnone%26legacyRegulation%3Dnone%26consentId%3Da80dd3f9-bb12-4a92-aa0b-67df3c229512%26analytical%3Dtrue%26marketing%3Dtrue; bkng_sso_auth=CAIQsOnuTRqEAfDLQZFUphkfekicBhBSmN8Qi+xnEi/1KJbA8+ejEH0tA+WTfJLEEW298miKYw+96xZ3QCKx9oaHsZ9l8uzrU84fy4fOkY1/qg3Q51WszH/0SoSJhjlqw9s3+4B+i4qPFvYTV0seBudrZkhSeJgPjxe+rn6GCneoOsMqw0nz4bI88FzF2Q==; bkng_sso_auth_1686197182=CAIQsOnuTRqEAfDLQZFUphkfekicBhBSmN8Qi+xnEi/1KJbA8+ejEH0tA+WTfJLEEW298miKYw+96xZ3QCKx9oaHsZ9l8uzrU84fy4fOkY1/qg3Q51WszH/0SoSJhjlqw9s3+4B+i4qPFvYTV0seBudrZkhSeJgPjxe+rn6GCneoOsMqw0nz4bI88FzF2Q==; BJS=-; _gid=GA1.2.1008496241.1686197186; _pxhd=GY5GoFE3eR2ccqOv5jULJxZxr%2FIzYAS8D8T7VGYAbiTP9XeFunyXIB9PfnHM7PKpvWk%2FnV5kvVP23H5CWtnD4A%3D%3D%3Am7S2O4GxunZp19XVpwK9cdFISZHpM3G4kcEQo45loVggtjE8HSmbKduF8FAKlirHJue%2FryBbpDf3lO7MvdumNN-WPyRrL%2FSjSQysiqdi8-E%3D; 11_srd=%7B%22features%22%3A%5B%7B%22id%22%3A9%7D%5D%2C%22score%22%3A3%2C%22detected%22%3Afalse%7D; g_state={"i_p":1686290930241,"i_l":2}; reese84=3:EIkaO3gYAqJ/yKE7JF3GFA==:OAeD8rMhSdwOh/U5uaiBW4tqtBKknVvdLB5lsT/rt1VgvMVXD7GJlEMkin4HRfemo5fiEyTuvTkuxgzoq1FfvCcLVeXMYe2LFyFm51s1OsWdaozxxcXjaSGGOkjU8C2E7IKvCn8kRp6/ztK+SH+gVHiWs14hsL12tHjW9oGXaczK8BelYNuzzUz+nttFwY+WD3sq9tAwro2l+7tqqtdTgXMiHiCR9mADEK9eJiUkBy8rPrdrD/Zb9bF9D70UFKLZEcljJ5cxSJ3jzMW8dlj1BrRzorphFHhDkQKNBDXaikolEi2VqEahQK0C62IsnHYQssVz8IcBKI2VPysCg+7S5gHFPztdU5sz6yOcXdDnouJnaBDNGOkGqbcagVnN6OogwJ26bIQoXygNlJ+KQgeqNh/68U3szVU6vCMO5FlVmDelnSxv2nsJciQcsc0fyCzWJaL9kqsPKwHauAoSAEyhYw==:uBUc/m+JCKXME7112v35xZFqKjCisjGkTldw3u2cF0I=; _pxff_tm=1; _screload=1; _pxff_cfp=1; _pxff_ddtc=1; _gat=1; lastSeen=0; _scid_r=3d7dbe98-21c5-466f-bc63-906fea9ac44f; _ga=GA1.1.1455947754.1685681565; _ga_FPD6YLJCJ7=GS1.1.1686204522.4.1.1686205526.0.0.0; _uetsid=d76816f005b111ee833d859bab60e724; _uetvid=525b6930010111ee839cc5c8b17ec0b6; _px3=2baa641c004634e78d21c2021b4d60a6d490191fb927fab7f806feda2a0bee6f:JRIvpB6xTZnkzsb7uAsF98Ab6u4KfjTJwMdPJTQtOxe2LDvM0vUAwPEaLv6rK4Jpi/36zq+VkyYQo6XERTt2qQ==:1000:PSp/AWAmSn+dIxYXlM54HNMqqhQN5/6JCFHAQu1+Ug9cr8SmjDNuDP4WDXFrlNkFii10XDfT9uUqcYj2G0VCUdzCiR925mqTma0RNK2IIxAeuGxHXeMMs3ri1pH6Qi3cuaDN7o9464cuNCxGgpiPmwXWXjCFs0viL7vuxuAO8D+mfPGI0aagxcbrcLPhifXPCKbUWAmceTjLvzGI5jrHmg==; _pxde=102ce6881adfac60559b7bc4f291def07951cfb25fdc8eee8b2a0a251b7c51cf:eyJ0aW1lc3RhbXAiOjE2ODYyMDU1Mjc5OTAsImZfa2IiOjAsImlwY19pZCI6W119; bkng=11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLbnmKTRaewPBu3bFsmPEGrc7HcBSRfMMnt1024U9HHZf%2FM9yrnReW9R6zAf5DLHCvN3VXADdyXAkHKdNrmDVM8FK3X2qaMR18V9R7h00KhzxFMOi85kbdMBjQqJWMpAnARKZaYleYyJZYol%2F7AUnVB3k5z2GpEdirdaqi02C5kiOA%3D'
  }
  response = requests.get(url, headers=headers)
  html = response.content
  soup = BeautifulSoup(html, "html.parser")

  property_cards = soup.find_all('div', class_='a826ba81c4 fa2f36ad22 afd256fc79 d08f526e0d ed11e24d01 ef9845d4b3 da89aeb942')
  for card in property_cards:
      title = card.find('div', class_='fcab3ed991 a23c043802').text.strip()
      price = int(card.find('span', class_='fcab3ed991 fbd1d3018c e729ed5ab6').text.strip().replace("US$", "").replace(",", "")) * 1302
      location = card.find('span', class_='f4bd0794db b4273d69aa').text.strip()
      URL_element = card.find('a', class_='fc63351294 a822bdf511 d4b6b7a9e7 fa565176a8 f7db01295e c334e6f658 f4605622ad b2f0d6a80e')
      URL = URL_element['href']

      cancel_element = card.find('div', class_='d506630cf3')
      cancel = cancel_element.text.strip() if cancel_element else ""

      rating_element = card.find('div', class_='b5cd09854e d10a6220b4')
      rating = rating_element.text.strip() if rating_element else ""

      overall_review_element = card.find('div', class_='b5cd09854e f0d4d6a2f5 e46e88563a')
      overall_review = overall_review_element.text.strip() if overall_review_element else ""

      review_num_element = card.find('div', class_='d8eab2cf7f c90c0a70d3 db63693c62')
      review_num = review_num_element.text.strip().replace("개 이용 후기", "").replace(",", "") if review_num_element else ""


      headers={
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
      'Cookie':'px_init=0; cors_js=1; pxcts=50fe2779-0101-11ee-8933-77634f496d71; _pxvid=50fe17fb-0101-11ee-8933-c6892157769a; bkng_sso_session=e30; OptanonConsent=implicitConsentCountry=nonGDPR&implicitConsentDate=1685681565899; _gcl_au=1.1.380996651.1685681568; bkng_prue=1; _pin_unauth=dWlkPU9HUTJOamcxTmpRdFpURXlOQzAwWm1NMUxXSXlOMll0TkRWbU5EUTRNak5rWkdGag; _scid=3d7dbe98-21c5-466f-bc63-906fea9ac44f; FPID=FPID2.2.nHLjNRnMk4vSNOlYGsPF33zdfmfU%2FML9mjj0o7FqZ2c%3D.1685681565; _sctr=1%7C1685631600000; _ga_A12345=GS1.1.1685681568.1.1.1685681648.0.0.0; bkng_sso_ses=eyJib29raW5nX2dsb2JhbCI6W3siaCI6ImVWSzVQRWwreHlyQkFlTW5hREpwSVlMOEVadG9rVDdScERwWUxpSGJ3ek0iLCJhIjoxfV19; pcm_consent=consentedAt%3D2023-06-08T04%3A06%3A22.650Z%26countryCode%3DKR%26expiresAt%3D2023-12-05T04%3A06%3A22.650Z%26implicit%3Dfalse%26regionCode%3D11%26regulation%3Dnone%26legacyRegulation%3Dnone%26consentId%3Da80dd3f9-bb12-4a92-aa0b-67df3c229512%26analytical%3Dtrue%26marketing%3Dtrue; bkng_sso_auth=CAIQsOnuTRqEAfDLQZFUphkfekicBhBSmN8Qi+xnEi/1KJbA8+ejEH0tA+WTfJLEEW298miKYw+96xZ3QCKx9oaHsZ9l8uzrU84fy4fOkY1/qg3Q51WszH/0SoSJhjlqw9s3+4B+i4qPFvYTV0seBudrZkhSeJgPjxe+rn6GCneoOsMqw0nz4bI88FzF2Q==; bkng_sso_auth_1686197182=CAIQsOnuTRqEAfDLQZFUphkfekicBhBSmN8Qi+xnEi/1KJbA8+ejEH0tA+WTfJLEEW298miKYw+96xZ3QCKx9oaHsZ9l8uzrU84fy4fOkY1/qg3Q51WszH/0SoSJhjlqw9s3+4B+i4qPFvYTV0seBudrZkhSeJgPjxe+rn6GCneoOsMqw0nz4bI88FzF2Q==; BJS=-; _gid=GA1.2.1008496241.1686197186; _pxhd=GY5GoFE3eR2ccqOv5jULJxZxr%2FIzYAS8D8T7VGYAbiTP9XeFunyXIB9PfnHM7PKpvWk%2FnV5kvVP23H5CWtnD4A%3D%3D%3Am7S2O4GxunZp19XVpwK9cdFISZHpM3G4kcEQo45loVggtjE8HSmbKduF8FAKlirHJue%2FryBbpDf3lO7MvdumNN-WPyRrL%2FSjSQysiqdi8-E%3D; 11_srd=%7B%22features%22%3A%5B%7B%22id%22%3A9%7D%5D%2C%22score%22%3A3%2C%22detected%22%3Afalse%7D; g_state={"i_p":1686290930241,"i_l":2}; reese84=3:EIkaO3gYAqJ/yKE7JF3GFA==:OAeD8rMhSdwOh/U5uaiBW4tqtBKknVvdLB5lsT/rt1VgvMVXD7GJlEMkin4HRfemo5fiEyTuvTkuxgzoq1FfvCcLVeXMYe2LFyFm51s1OsWdaozxxcXjaSGGOkjU8C2E7IKvCn8kRp6/ztK+SH+gVHiWs14hsL12tHjW9oGXaczK8BelYNuzzUz+nttFwY+WD3sq9tAwro2l+7tqqtdTgXMiHiCR9mADEK9eJiUkBy8rPrdrD/Zb9bF9D70UFKLZEcljJ5cxSJ3jzMW8dlj1BrRzorphFHhDkQKNBDXaikolEi2VqEahQK0C62IsnHYQssVz8IcBKI2VPysCg+7S5gHFPztdU5sz6yOcXdDnouJnaBDNGOkGqbcagVnN6OogwJ26bIQoXygNlJ+KQgeqNh/68U3szVU6vCMO5FlVmDelnSxv2nsJciQcsc0fyCzWJaL9kqsPKwHauAoSAEyhYw==:uBUc/m+JCKXME7112v35xZFqKjCisjGkTldw3u2cF0I=; _pxff_tm=1; _screload=1; _pxff_cfp=1; _pxff_ddtc=1; _gat=1; lastSeen=0; _scid_r=3d7dbe98-21c5-466f-bc63-906fea9ac44f; _ga=GA1.1.1455947754.1685681565; _ga_FPD6YLJCJ7=GS1.1.1686204522.4.1.1686205526.0.0.0; _uetsid=d76816f005b111ee833d859bab60e724; _uetvid=525b6930010111ee839cc5c8b17ec0b6; _px3=2baa641c004634e78d21c2021b4d60a6d490191fb927fab7f806feda2a0bee6f:JRIvpB6xTZnkzsb7uAsF98Ab6u4KfjTJwMdPJTQtOxe2LDvM0vUAwPEaLv6rK4Jpi/36zq+VkyYQo6XERTt2qQ==:1000:PSp/AWAmSn+dIxYXlM54HNMqqhQN5/6JCFHAQu1+Ug9cr8SmjDNuDP4WDXFrlNkFii10XDfT9uUqcYj2G0VCUdzCiR925mqTma0RNK2IIxAeuGxHXeMMs3ri1pH6Qi3cuaDN7o9464cuNCxGgpiPmwXWXjCFs0viL7vuxuAO8D+mfPGI0aagxcbrcLPhifXPCKbUWAmceTjLvzGI5jrHmg==; _pxde=102ce6881adfac60559b7bc4f291def07951cfb25fdc8eee8b2a0a251b7c51cf:eyJ0aW1lc3RhbXAiOjE2ODYyMDU1Mjc5OTAsImZfa2IiOjAsImlwY19pZCI6W119; bkng=11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLbnmKTRaewPBu3bFsmPEGrc7HcBSRfMMnt1024U9HHZf%2FM9yrnReW9R6zAf5DLHCvN3VXADdyXAkHKdNrmDVM8FK3X2qaMR18V9R7h00KhzxFMOi85kbdMBjQqJWMpAnARKZaYleYyJZYol%2F7AUnVB3k5z2GpEdirdaqi02C5kiOA%3D'
      }
      response = requests.get(URL, headers=headers)
      html = response.content
      soup_tmp = BeautifulSoup(html, "html.parser")

      address = soup_tmp.find('span', class_='hp_address_subtitle js-hp_address_subtitle jq_tooltip').text.strip()
      ptype = soup_tmp.find('span', class_='bui-button__text').text.strip()

      review_cards = soup_tmp.find_all('div', class_='ccff2b4c43 cfc0860887')
      detailed_review_ratings = []
      for review_card in review_cards:
          detailed_review_rating_element = review_card.find('div', class_='ee746850b6 b8eef6afe1')
          detailed_review_rating = detailed_review_rating_element.text.strip() if detailed_review_rating_element else ""
          detailed_review_ratings.append(detailed_review_rating)

      if not detailed_review_ratings or len(detailed_review_ratings) < 7:
        detailed_review_ratings = [""] * 7

      data.append({
          "Title": title,
          "Type" : ptype,
          "Location": location,
          "Address": address,
          "Price": price,
          "Cancellation Policy": cancel,
          "Review Number": review_num,
          "Overall Review": overall_review,
          "Rating": rating,
          "StaffFriendlinessR" : detailed_review_ratings[0],
          "Facilities" : detailed_review_ratings[1],
          "CleanlinessR" : detailed_review_ratings[2],
          "ComfortR" : detailed_review_ratings[3],
          "Cost-effectivenessR" : detailed_review_ratings[4],
          "LocationR" : detailed_review_ratings[5],
          "Free Wi-FiR" : detailed_review_ratings[6],
          "URL": URL,

      })

print()
print('done')
df = pd.DataFrame(data)

1page    - - - - - ing

2page    - - - - - ing

3page    - - - - - ing

4page    - - - - - ing

5page    - - - - - ing

6page    - - - - - ing

7page    - - - - - ing

8page    - - - - - ing

9page    - - - - - ing

10page    - - - - - ing

11page    - - - - - ing

12page    - - - - - ing


done


In [ ]:
df

,Title,Type,Location,Address,Price,Cancellation Policy,Review Number,Overall Review,Rating,StaffFriendlinessR,Facilities,CleanlinessR,ComfortR,Cost-effectivenessR,LocationR,Free Wi-FiR,URL
0,롯데시티호텔 제주,호텔,"제주 시내, 제주","제주, 도령로 83",395808,무료취소,1677,매우 좋음,8.4,9.1,8.8,8.9,9.0,8.3,9.0,8.9,https://www.booking.com/hotel/kr/lotte-city-je...
1,오션스위츠 제주 호텔,호텔,"제주 시내, 제주","제주, 탑동 해안로 74",209622,무료취소,1131,좋음,7.9,8.8,8.1,8.2,8.3,8.2,9.0,8.7,https://www.booking.com/hotel/kr/ocean-suites-...
2,라마다 제주시티호텔,호텔,"제주 시내, 제주","제주, 중앙로 304",218736,무료취소,2696,매우 좋음,8.0,8.6,8.3,8.4,8.3,8.7,8.6,8.8,https://www.booking.com/hotel/kr/ramada-jeju-c...
3,호텔 시리우스,호텔,"제주 시내, 제주","제주, 도령로 133",208320,무료취소,1045,매우 좋음,8.2,8.6,8.5,8.6,8.5,8.5,8.8,8.8,https://www.booking.com/hotel/kr/sirius.ko.htm...
4,제주 아길라 호텔 오시아노 스위츠,호텔,"한림읍, 제주","제주, 한림읍 옹포2길 10",359352,무료취소,618,매우 좋음,8.0,8.3,8.2,8.3,8.4,7.9,8.4,8.3,https://www.booking.com/hotel/kr/aguila.ko.htm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,젠틀리조트,리조트,"한경면, 제주","제주, 한경면 일주서로 4010-2",242172,무료취소,외부 사이트 후기 6개,최고 9.0,,,,,,,,,https://www.booking.com/hotel/kr/gentle-resort...
254,아프리카 게스트하우스,게스트하우스,"조천읍, 제주","제주, 조천읍 신흥로2길 33 (조천읍)",92442,무료취소,22,좋음,7.9,9.1,7.0,7.5,7.3,8.6,8.7,9.1,https://www.booking.com/hotel/kr/apeurikageseu...
255,예스준 게스트하우스,게스트하우스,"제주 시내, 제주","제주, 서해안로 232",108066,무료취소,28,좋음,7.9,8.4,7.9,7.8,7.5,8.1,8.3,5.0,https://www.booking.com/hotel/kr/yesjun-guest-...
256,바오젠 게스트하우스,게스트하우스,"제주 시내, 제주","제주, 은남2길 41 3층",108066,무료취소,20,좋음,7.6,8.6,7.6,7.6,8.0,8.0,8.1,9.4,https://www.booking.com/hotel/kr/baozen-guesth...


In [ ]:
# 웹 크롤링 결과 저장 (경로 : 예시)
df.to_csv('booking_dot_com_data.csv', index=False)

# 테이블 생성 위한 데이터 처리

## 데이터 불러오기

In [ ]:
# csv 불러오기 (경로 : 예시)
bookingdata = pd.read_csv('booking_dot_com_data.csv')

In [ ]:
# 데이터 확인
bookingdata.head(35)

,Title,Type,Location,Address,Price,Cancellation Policy,Review Number,Overall Review,Rating,StaffFriendlinessR,Facilities,CleanlinessR,ComfortR,Cost-effectivenessR,LocationR,Free Wi-FiR,URL
0,롯데시티호텔 제주,호텔,"제주 시내, 제주","제주, 도령로 83",395808,무료취소,1677,매우 좋음,8.4,9.1,8.8,8.9,9.0,8.3,9.0,8.9,https://www.booking.com/hotel/kr/lotte-city-jeju.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=89448612_134358957_2_42_0&highlighted_blocks=89448612_134358957_2_42_0&matching_block_id=89448612_134358957_2_42_0&sr_pri_blocks=89448612_134358957_2_42_0__39600000&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
1,오션스위츠 제주 호텔,호텔,"제주 시내, 제주","제주, 탑동 해안로 74",209622,무료취소,1131,좋음,7.9,8.8,8.1,8.2,8.3,8.2,9.0,8.7,https://www.booking.com/hotel/kr/ocean-suites-jeju-jeju.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=101479753_190176311_2_2_0&highlighted_blocks=101479753_190176311_2_2_0&matching_block_id=101479753_190176311_2_2_0&sr_pri_blocks=101479753_190176311_2_2_0__21000000&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
2,라마다 제주시티호텔,호텔,"제주 시내, 제주","제주, 중앙로 304",218736,무료취소,2696,매우 좋음,8.0,8.6,8.3,8.4,8.3,8.7,8.6,8.8,https://www.booking.com/hotel/kr/ramada-jeju-city-hall.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=0_0_2_0_0&highlighted_blocks=0_0_2_0_0&matching_block_id=0_0_2_0_0&sr_pri_blocks=0_0_2_0_0__21961638&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
3,호텔 시리우스,호텔,"제주 시내, 제주","제주, 도령로 133",208320,무료취소,1045,매우 좋음,8.2,8.6,8.5,8.6,8.5,8.5,8.8,8.8,https://www.booking.com/hotel/kr/sirius.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=4&hapos=4&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=318899301_0_2_0_0&highlighted_blocks=318899301_0_2_0_0&matching_block_id=318899301_0_2_0_0&sr_pri_blocks=318899301_0_2_0_0__20887301&from=searchresults#hotelTmpl
4,제주 아길라 호텔 오시아노 스위츠,호텔,"한림읍, 제주","제주, 한림읍 옹포2길 10",359352,무료취소,618,매우 좋음,8.0,8.3,8.2,8.3,8.4,7.9,8.4,8.3,https://www.booking.com/hotel/kr/aguila.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=5&hapos=5&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=202191902_333996229_2_0_0&highlighted_blocks=202191902_333996229_2_0_0&matching_block_id=202191902_333996229_2_0_0&sr_pri_blocks=202191902_333996229_2_0_0__360

## 데이터 확인하기

In [ ]:
bookingdata.isna().sum()
# 결측치 처리 필요
# Cancallation Policy : X (O/X로 수정)
# Overall Review : 신규 숙소
# 리뷰들 : 0

Title                   0
Type                    0
Location                0
Address                 0
Price                   0
Cancellation Policy    38
Review Number          22
Overall Review         22
Rating                 30
StaffFriendlinessR     31
Facilities             31
CleanlinessR           31
ComfortR               31
Cost-effectivenessR    31
LocationR              32
Free Wi-FiR            31
URL                     0
dtype: int64

In [ ]:
bookingdata.dtypes
# review Number : object -> 텍스트 제거 후 int로 변경

Title                   object
Type                    object
Location                object
Address                 object
Price                    int64
Cancellation Policy     object
Review Number           object
Overall Review          object
Rating                 float64
StaffFriendlinessR     float64
Facilities             float64
CleanlinessR           float64
ComfortR               float64
Cost-effectivenessR    float64
LocationR              float64
Free Wi-FiR            float64
URL                     object
dtype: object

+) Location, Overall Review -> 유일값 찾아 일반화하기

## 결측치 처리하기

In [ ]:
# 무료 취소 가능 여부 O/X로 수정
bookingdata['Cancellation Policy'] = bookingdata['Cancellation Policy'].apply(lambda x: 'O' if x == '무료취소' else 'X')

In [ ]:
# Overall Review가 결측치일 경우, '신규 숙소'로 수정
bookingdata.loc[bookingdata['Overall Review'].isnull(), 'Overall Review'] = '신규 숙소'

In [ ]:
bookingdata = bookingdata.fillna(0)
# 나머지 결측치 0으로 채우기

In [ ]:
# 데이터 확인 -> 처리 성공
bookingdata.head(35)

,Title,Type,Location,Address,Price,Cancellation Policy,Review Number,Overall Review,Rating,StaffFriendlinessR,Facilities,CleanlinessR,ComfortR,Cost-effectivenessR,LocationR,Free Wi-FiR,URL
0,롯데시티호텔 제주,호텔,"제주 시내, 제주","제주, 도령로 83",395808,O,1677,매우 좋음,8.4,9.1,8.8,8.9,9.0,8.3,9.0,8.9,https://www.booking.com/hotel/kr/lotte-city-jeju.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=89448612_134358957_2_42_0&highlighted_blocks=89448612_134358957_2_42_0&matching_block_id=89448612_134358957_2_42_0&sr_pri_blocks=89448612_134358957_2_42_0__39600000&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
1,오션스위츠 제주 호텔,호텔,"제주 시내, 제주","제주, 탑동 해안로 74",209622,O,1131,좋음,7.9,8.8,8.1,8.2,8.3,8.2,9.0,8.7,https://www.booking.com/hotel/kr/ocean-suites-jeju-jeju.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=101479753_190176311_2_2_0&highlighted_blocks=101479753_190176311_2_2_0&matching_block_id=101479753_190176311_2_2_0&sr_pri_blocks=101479753_190176311_2_2_0__21000000&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
2,라마다 제주시티호텔,호텔,"제주 시내, 제주","제주, 중앙로 304",218736,O,2696,매우 좋음,8.0,8.6,8.3,8.4,8.3,8.7,8.6,8.8,https://www.booking.com/hotel/kr/ramada-jeju-city-hall.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=0_0_2_0_0&highlighted_blocks=0_0_2_0_0&matching_block_id=0_0_2_0_0&sr_pri_blocks=0_0_2_0_0__21961638&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
3,호텔 시리우스,호텔,"제주 시내, 제주","제주, 도령로 133",208320,O,1045,매우 좋음,8.2,8.6,8.5,8.6,8.5,8.5,8.8,8.8,https://www.booking.com/hotel/kr/sirius.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=4&hapos=4&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=318899301_0_2_0_0&highlighted_blocks=318899301_0_2_0_0&matching_block_id=318899301_0_2_0_0&sr_pri_blocks=318899301_0_2_0_0__20887301&from=searchresults#hotelTmpl
4,제주 아길라 호텔 오시아노 스위츠,호텔,"한림읍, 제주","제주, 한림읍 옹포2길 10",359352,O,618,매우 좋음,8.0,8.3,8.2,8.3,8.4,7.9,8.4,8.3,https://www.booking.com/hotel/kr/aguila.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=5&hapos=5&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=202191902_333996229_2_0_0&highlighted_blocks=202191902_333996229_2_0_0&matching_block_id=202191902_333996229_2_0_0&sr_pri_blocks=202191902_333996229_2_0_0__36000000&from_sust

## 데이터타입 수정

In [ ]:
bookingdata['Review Number'].unique()
#-> '외부 사이트 후기 n개'의 형식 존재

array(['1677', '1131', '2696', '1045', '618', '466', '1419', '1946',
       '1833', '1211', '437', '1663', '1080', '796', '118', '1069', '845',
       '608', '1210', '789', '1465', '1', '86', '240', '64', '47', '35',
       '169', 0, '467', '560', '1514', '498', '573', '504', '127', '9',
       '17', '479', '162', '247', '7', '28', '773', '304', '83', '48',
       '206', '117', '2', '40', '379', '16', '401', '375', '45', '131',
       '65', '471', '80', '364', '1208', '12', '152', '223', '59', '607',
       '32', '370', '22', '102', '936', '211', '336', '49', '69', '112',
       '201', '62', '41', '93', '126', '184', '114', '75', '14', '11',
       '34', '81', '543', '123', '5', '78', '311', '182', '95', '173',
       '26', '366', '273', '158', '104', '153', '60', '87', '71', '25',
       '631', '38', '외부 사이트 후기 4개', '외부 사이트 후기 40개', '4', '10', '79',
       '183', '8', '50', '18', '31', '23', '926', '242', '125', '15',
       '13', '44', '73', '56', '43', '33', '20', '76', '294', '230'

In [ ]:
import re

bookingdata['Review Number'] = bookingdata['Review Number'].str.extract('(\d+)', expand=False).fillna(0).astype(int)
bookingdata['Review Number'].unique()

array([1677, 1131, 2696, 1045,  618,  466, 1419, 1946, 1833, 1211,  437,
       1663, 1080,  796,  118, 1069,  845,  608, 1210,  789, 1465,    1,
         86,  240,   64,   47,   35,  169,    0,  467,  560, 1514,  498,
        573,  504,  127,    9,   17,  479,  162,  247,    7,   28,  773,
        304,   83,   48,  206,  117,    2,   40,  379,   16,  401,  375,
         45,  131,   65,  471,   80,  364, 1208,   12,  152,  223,   59,
        607,   32,  370,   22,  102,  936,  211,  336,   49,   69,  112,
        201,   62,   41,   93,  126,  184,  114,   75,   14,   11,   34,
         81,  543,  123,    5,   78,  311,  182,   95,  173,   26,  366,
        273,  158,  104,  153,   60,   87,   71,   25,  631,   38,    4,
         10,   79,  183,    8,   50,   18,   31,   23,  926,  242,  125,
         15,   13,   44,   73,   56,   43,   33,   20,   76,  294,  230,
        148,    3,  146,   42,    6,  233,  250,  248,   29,  279])

## Location

In [ ]:
print(bookingdata['Location'].unique())

['제주 시내, 제주' '한림읍, 제주' '제주' '애월읍, 제주' '구좌읍, 제주' '우도, 제주' '조천읍, 제주'
 '한경면, 제주']


In [ ]:
jeju_rows = bookingdata[bookingdata['Location'] == '제주']
pd.set_option('display.max_colwidth', None)  # URL 행이 잘리지 않도록 컬럼 너비 설정
print(len(jeju_rows))
print(jeju_rows['URL'])
# 16개의 기타 도시 존재

16
8                                                                                                    https://www.booking.com/hotel/kr/nanta-jeju.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=9&hapos=9&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=213857401_0_2_0_0&highlighted_blocks=213857401_0_2_0_0&matching_block_id=213857401_0_2_0_0&sr_pri_blocks=213857401_0_2_0_0__21150658&from=searchresults#hotelTmpl
27                                                          https://www.booking.com/hotel/kr/haneultaripensyeon.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4Aqi_h6QGwAIB0gIkOTExMDcxYjgtNzEzZC00YTVk

In [ ]:
bookingdata['Location'] = bookingdata['Location'].str.replace(', 제주', '')
bookingdata.loc[bookingdata['Location'] == '제주', 'Location'] = '기타 도시'

In [ ]:
print(bookingdata['Location'].unique())

['제주 시내' '한림읍' '기타 도시' '애월읍' '구좌읍' '우도' '조천읍' '한경면']


## Overall Review

In [ ]:
print(bookingdata['Overall Review'].unique())

['매우 좋음' '좋음' '우수함' '최고' '강력 추천' '신규 숙소' '이용 후기 평점' '강력 추천 10' '최고 9.0']


In [ ]:
# 해당 값의 Rating 범위 확인
overall_reviews = bookingdata['Overall Review'].unique()

for review in overall_reviews:
    rating_range = bookingdata.loc[bookingdata['Overall Review'] == review, 'Rating']
    rating_min = rating_range.min()
    rating_max = rating_range.max()

    print(f"{review}일 때의 Rating 최소값: {rating_min}")
    print(f"{review}일 때의 Rating 최대값: {rating_max}")
    print()

매우 좋음일 때의 Rating 최소값: 8.0
매우 좋음일 때의 Rating 최대값: 8.5

좋음일 때의 Rating 최소값: 7.0
좋음일 때의 Rating 최대값: 7.9

우수함일 때의 Rating 최소값: 8.6
우수함일 때의 Rating 최대값: 8.9

최고일 때의 Rating 최소값: 9.0
최고일 때의 Rating 최대값: 9.4

강력 추천일 때의 Rating 최소값: 9.5
강력 추천일 때의 Rating 최대값: 10.0

신규 숙소일 때의 Rating 최소값: 0.0
신규 숙소일 때의 Rating 최대값: 0.0

이용 후기 평점일 때의 Rating 최소값: 1.0
이용 후기 평점일 때의 Rating 최대값: 6.9

강력 추천 10일 때의 Rating 최소값: 0.0
강력 추천 10일 때의 Rating 최대값: 0.0

최고 9.0일 때의 Rating 최소값: 0.0
최고 9.0일 때의 Rating 최대값: 0.0



어딘가 이상함을 알 수 있음

강력 추천 10, 최고 9.0, 이용 후기 평점 처리 필요!

-> 강력 추천 10/최고 9.0 -> 리뷰 X 신규 숙소로 처리

-> 이용 후기 평점 : 조금 더 세세하게 나누기
    
      0 ~ 5 : 실망스러움 / 5 ~ 6 보통 / 6 ~ 7 괜찮

In [ ]:
print(bookingdata['Rating'].unique())

[ 8.4  7.9  8.   8.2  8.1  8.6  7.8  7.7  8.9  7.5  8.8  9.   8.3  9.5
  0.   7.2  7.4  7.   7.3  6.9  8.5 10.   8.7  7.1  9.2  7.6  6.6  6.8
  5.6  9.1  6.7  9.6  1.   6.5  9.4  5.8  4.   6.3  5.3  5.5  6.1  9.7
  3.6  6.   5.   5.7]


In [ ]:
# 강력 추천 10과 최고 9.0을 '신규 숙소'로 처리
bookingdata.loc[(bookingdata['Overall Review'] == '강력 추천 10') | (bookingdata['Overall Review'] == '최고 9.0'), 'Overall Review'] = '신규 숙소'

# 이용 후기 평점 범위에 따라 세부 처리
bookingdata.loc[bookingdata['Overall Review'] == '이용 후기 평점', 'Overall Review'] = pd.cut(bookingdata.loc[bookingdata['Overall Review'] == '이용 후기 평점', 'Rating'], bins=[0, 4.99, 5.99, 6.99], labels=['실망스러움', '보통', '괜찮음'])

# 처리 결과 출력
print(bookingdata['Overall Review'].unique())

['매우 좋음' '좋음' '우수함' '최고' '강력 추천' '신규 숙소' '괜찮음' '보통' '실망스러움']


In [ ]:
# 해당 값의 Rating 범위 확인
overall_reviews = bookingdata['Overall Review'].unique()

for review in overall_reviews:
    rating_range = bookingdata.loc[bookingdata['Overall Review'] == review, 'Rating']
    rating_min = rating_range.min()
    rating_max = rating_range.max()

    print(f"{review}일 때의 Rating 최소값: {rating_min}")
    print(f"{review}일 때의 Rating 최대값: {rating_max}")
    print()

매우 좋음일 때의 Rating 최소값: 8.0
매우 좋음일 때의 Rating 최대값: 8.5

좋음일 때의 Rating 최소값: 7.0
좋음일 때의 Rating 최대값: 7.9

우수함일 때의 Rating 최소값: 8.6
우수함일 때의 Rating 최대값: 8.9

최고일 때의 Rating 최소값: 9.0
최고일 때의 Rating 최대값: 9.4

강력 추천일 때의 Rating 최소값: 9.5
강력 추천일 때의 Rating 최대값: 10.0

신규 숙소일 때의 Rating 최소값: 0.0
신규 숙소일 때의 Rating 최대값: 0.0

괜찮음일 때의 Rating 최소값: 6.0
괜찮음일 때의 Rating 최대값: 6.9

보통일 때의 Rating 최소값: 5.0
보통일 때의 Rating 최대값: 5.8

실망스러움일 때의 Rating 최소값: 1.0
실망스러움일 때의 Rating 최대값: 4.0



-> 현재의 범위

[0,5) : 실망스러움

[5,6) : 보통

[6,7) : 괜찮음

[7,8) : 좋음

[8,8.5] : 매우 좋음

(8.5, 9) : 우수함

[9, 9.5) : 최고

[9.5, 10) : 강력 추천

-> 좋음 ~ 매우 좋음에서, 8.5가 범위를 꼬이게 한다는 것 발견

In [ ]:
# 8.5를 우수함으로 !
bookingdata.loc[bookingdata['Rating'] == 8.5, 'Overall Review'] = '우수함'

In [ ]:
# 해당 값의 Rating 범위 확인
overall_reviews = bookingdata['Overall Review'].unique()

for review in overall_reviews:
    rating_range = bookingdata.loc[bookingdata['Overall Review'] == review, 'Rating']
    rating_min = rating_range.min()
    rating_max = rating_range.max()

    print(f"{review}일 때의 Rating 최소값: {rating_min}")
    print(f"{review}일 때의 Rating 최대값: {rating_max}")
    print()

매우 좋음일 때의 Rating 최소값: 8.0
매우 좋음일 때의 Rating 최대값: 8.4

좋음일 때의 Rating 최소값: 7.0
좋음일 때의 Rating 최대값: 7.9

우수함일 때의 Rating 최소값: 8.5
우수함일 때의 Rating 최대값: 8.9

최고일 때의 Rating 최소값: 9.0
최고일 때의 Rating 최대값: 9.4

강력 추천일 때의 Rating 최소값: 9.5
강력 추천일 때의 Rating 최대값: 10.0

신규 숙소일 때의 Rating 최소값: 0.0
신규 숙소일 때의 Rating 최대값: 0.0

괜찮음일 때의 Rating 최소값: 6.0
괜찮음일 때의 Rating 최대값: 6.9

보통일 때의 Rating 최소값: 5.0
보통일 때의 Rating 최대값: 5.8

실망스러움일 때의 Rating 최소값: 1.0
실망스러움일 때의 Rating 최대값: 4.0



-> 보기 좋게 정리된 범위를 확인할 수 있음

[0,5) : 실망스러움

[5,6) : 보통

[6,7) : 괜찮음

[7,8) : 좋음

[8,8.5) : 매우 좋음

[8.5, 9) : 우수함

[9, 9.5) : 최고

[9.5, 10) : 강력 추천

## 중복 데이터 처리하기

In [ ]:
len(bookingdata)

258

In [ ]:
# 중복된 Title 개수 / 내용 확인
duplicates = bookingdata[bookingdata.duplicated(subset='Title', keep=False)]
duplicates = duplicates.sort_values('Title')
print("중복 Title 개수:", len(duplicates))
duplicates

중복 Title 개수: 28


,Title,Type,Location,Address,Price,Cancellation Policy,Review Number,Overall Review,Rating,StaffFriendlinessR,Facilities,CleanlinessR,ComfortR,Cost-effectivenessR,LocationR,Free Wi-FiR,URL
32,그림리조트,리조트,제주 시내,"제주, 서해안로 620-1",195300,X,560,좋음,7.2,8.5,7.4,7.5,7.6,7.7,8.2,8.3,https://www.booking.com/hotel/kr/grim-resort.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4Aqi_h6QGwAIB0gIkOTExMDcxYjgtNzEzZC00YTVkLTgwM2MtMGY0NzEyNDE2Njg12AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=8&hapos=33&sr_order=popularity&srpvid=179062d414fe04a5&srepoch=1686233002&all_sr_blocks=223877912_332195106_2_0_0&highlighted_blocks=223877912_332195106_2_0_0&matching_block_id=223877912_332195106_2_0_0&sr_pri_blocks=223877912_332195106_2_0_0__19620000&from=searchresults#hotelTmpl
82,그림리조트,리조트,제주 시내,"제주, 서해안로 620-1",195300,X,560,좋음,7.2,8.5,7.4,7.5,7.6,7.7,8.2,8.3,https://www.booking.com/hotel/kr/grim-resort.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AtDAh6QGwAIB0gIkMGJmOTIyODUtZTliZS00MjFjLTlkMzAtOGJhMzEwM2EyNDVi2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=8&hapos=83&sr_order=popularity&srpvid=5f3663281a1e015e&srepoch=1686233169&all_sr_blocks=223877912_332195106_2_0_0&highlighted_blocks=223877912_332195106_2_0_0&matching_block_id=223877912_332195106_2_0_0&sr_pri_blocks=223877912_332195106_2_0_0__19620000&from=searchresults#hotelTmpl
6,글로스터 호텔 제주,호텔,제주 시내,"제주, 삼무로 57",145824,O,1419,매우 좋음,8.2,8.7,8.3,8.5,8.5,8.7,9.0,8.7,https://www.booking.com/hotel/kr/howard-johnson-jeju-city.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=7&hapos=7&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=222454029_329565677_0_2_0&highlighted_blocks=222454029_329565677_0_2_0&matching_block_id=222454029_329565677_0_2_0&sr_pri_blocks=222454029_329565677_0_2_0__14620500&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
31,글로스터 호텔 제주,호텔,제주 시내,"제주, 삼무로 57",145824,O,1419,매우 좋음,8.2,8.7,8.3,8.5,8.5,8.7,9.0,8.7,https://www.booking.com/hotel/kr/howard-johnson-jeju-city.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4Aqi_h6QGwAIB0gIkOTExMDcxYjgtNzEzZC00YTVkLTgwM2MtMGY0NzEyNDE2Njg12AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=7&hapos=32&sr_order=popularity&srpvid=179062d414fe04a5&srepoch=1686233002&all_sr_blocks=222454029_329565677_0_2_0&highlighted_blocks=222454029_329565677_0_2_0&matching_block_id=222454029_329565677_0_2_0&sr_pri_blocks=222454029_329565677_0_2_0__14620500&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
49,길리 리조트,리조트,한림읍,"제주, 한림읍 명재로 100-47",459606,O,0,신규 숙소,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.booking.com/hotel/kr/gili-resort.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4Aqi_h6QGwAIB0gIkOTExMDcxYjgtNzEzZC00YTVkLTgwM2MtMGY0NzEyNDE2Njg12AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=25&hapos=50&sr_order=popularity&srpvid=179062d414fe04a5&srepoch=1686233002&all_sr_blocks=996391901_372776306_2_0_0&highlighted_blocks=996391901_372776306_2_0_0&matching_block_id=996391901_372776306_2_0_0&sr_pri_blocks=9963919

In [ ]:
# 해당 Title의 첫째 행만 남기기 (딱 두 행의 미미한 가격차이 제외하고는 다른 정보들 모두 동일, 일반화를 위해 중복행 제거처리)
bookingdata.drop_duplicates(subset='Title', keep='first', inplace=True)

In [ ]:
# 14개의 행이 사라진 것을 확인할 수 있음
len(bookingdata)

244

## 데이터 결과 확인하기

In [ ]:
bookingdata.dtypes
# 모두 알맞은 데이터타입

Title                   object
Type                    object
Location                object
Address                 object
Price                    int64
Cancellation Policy     object
Review Number            int64
Overall Review          object
Rating                 float64
StaffFriendlinessR     float64
Facilities             float64
CleanlinessR           float64
ComfortR               float64
Cost-effectivenessR    float64
LocationR              float64
Free Wi-FiR            float64
URL                     object
dtype: object

In [ ]:
bookingdata.isna().sum()
# 모든 결측치 0

Title                  0
Type                   0
Location               0
Address                0
Price                  0
Cancellation Policy    0
Review Number          0
Overall Review         0
Rating                 0
StaffFriendlinessR     0
Facilities             0
CleanlinessR           0
ComfortR               0
Cost-effectivenessR    0
LocationR              0
Free Wi-FiR            0
URL                    0
dtype: int64

In [ ]:
# 중복된 Title 개수 없음 -> key값 사용 가능 !
duplicates = bookingdata[bookingdata.duplicated(subset='Title', keep=False)]
duplicates = duplicates.sort_values('Title')
print("중복 Title 개수:", len(duplicates))

중복 Title 개수: 0


In [ ]:
bookingdata.head(50)

,Title,Type,Location,Address,Price,Cancellation Policy,Review Number,Overall Review,Rating,StaffFriendlinessR,Facilities,CleanlinessR,ComfortR,Cost-effectivenessR,LocationR,Free Wi-FiR,URL
0,롯데시티호텔 제주,호텔,제주 시내,"제주, 도령로 83",395808,O,1677,매우 좋음,8.4,9.1,8.8,8.9,9.0,8.3,9.0,8.9,https://www.booking.com/hotel/kr/lotte-city-jeju.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=89448612_134358957_2_42_0&highlighted_blocks=89448612_134358957_2_42_0&matching_block_id=89448612_134358957_2_42_0&sr_pri_blocks=89448612_134358957_2_42_0__39600000&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
1,오션스위츠 제주 호텔,호텔,제주 시내,"제주, 탑동 해안로 74",209622,O,1131,좋음,7.9,8.8,8.1,8.2,8.3,8.2,9.0,8.7,https://www.booking.com/hotel/kr/ocean-suites-jeju-jeju.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=101479753_190176311_2_2_0&highlighted_blocks=101479753_190176311_2_2_0&matching_block_id=101479753_190176311_2_2_0&sr_pri_blocks=101479753_190176311_2_2_0__21000000&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
2,라마다 제주시티호텔,호텔,제주 시내,"제주, 중앙로 304",218736,O,2696,매우 좋음,8.0,8.6,8.3,8.4,8.3,8.7,8.6,8.8,https://www.booking.com/hotel/kr/ramada-jeju-city-hall.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=0_0_2_0_0&highlighted_blocks=0_0_2_0_0&matching_block_id=0_0_2_0_0&sr_pri_blocks=0_0_2_0_0__21961638&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
3,호텔 시리우스,호텔,제주 시내,"제주, 도령로 133",208320,O,1045,매우 좋음,8.2,8.6,8.5,8.6,8.5,8.5,8.8,8.8,https://www.booking.com/hotel/kr/sirius.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=4&hapos=4&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=318899301_0_2_0_0&highlighted_blocks=318899301_0_2_0_0&matching_block_id=318899301_0_2_0_0&sr_pri_blocks=318899301_0_2_0_0__20887301&from=searchresults#hotelTmpl
4,제주 아길라 호텔 오시아노 스위츠,호텔,한림읍,"제주, 한림읍 옹포2길 10",359352,O,618,매우 좋음,8.0,8.3,8.2,8.3,8.4,7.9,8.4,8.3,https://www.booking.com/hotel/kr/aguila.ko.html?aid=304142&label=gen173nr-1FCAQoggJCDGNpdHlfLTcxNDE5MUgXWARopgKIAQGYARe4ARjIAQzYAQHoAQH4AQOIAgGoAgS4AsC-h6QGwAIB0gIkYWYxNTEwM2ItYzZhYy00MzlmLTg4NDMtZjQzOGE4MGM3ZmJh2AIF4AIB&ucfs=1&arphpl=1&checkin=2023-07-31&checkout=2023-08-02&dest_id=-714191&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=5&hapos=5&sr_order=popularity&srpvid=c55a62a00dac0037&srepoch=1686232898&all_sr_blocks=202191902_333996229_2_0_0&highlighted_blocks=202191902_333996229_2_0_0&matching_block_id=202191902_333996229_2_0_0&sr_pri_blocks=202191902_333996229_2_0_0__36000000&from_sustainable_property_sr=1&from=sea

## 테이블을 위한 csv파일 생성

In [ ]:
bookingdata.columns

Index(['Title', 'Type', 'Location', 'Address', 'Price', 'Cancellation Policy',
       'Review Number', 'Overall Review', 'Rating', 'StaffFriendlinessR',
       'Facilities', 'CleanlinessR', 'ComfortR', 'Cost-effectivenessR',
       'LocationR', 'Free Wi-FiR', 'URL'],
      dtype='object')

In [ ]:
# 테이블에 필요한 컬럼 리스트
basic_info = [
    'Title', 'Type', 'Location', 'Price'
    ]
detail_info = [
    'Title', 'Address', 'Price', 'Cancellation Policy', 'Review Number', 'Overall Review', 'URL'
    ]
review_info = [
    'Title', 'Review Number', 'Overall Review', 'Rating', 'StaffFriendlinessR',
    'Facilities', 'CleanlinessR', 'ComfortR', 'Cost-effectivenessR',
    'LocationR', 'Free Wi-FiR'
    ]

In [ ]:
# 테이블을 위한 필요한 컬럼의 내용만 불러온 df 생서
basic_info_df = bookingdata[basic_info].copy()
detail_info_df = bookingdata[detail_info].copy()
review_info_df = bookingdata[review_info].copy()

In [ ]:
# CSV 파일로 저장 (경로 : 예시)
basic_info_df.to_csv('basic_info.csv')
detail_info_df.to_csv('detail_info.csv')
review_info_df.to_csv('review_info.csv')